In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets.cifar100 import load_data

In [0]:
(x_train,y_train),(x_test,y_test) = load_data()

In [41]:
print(x_train.max())

255


In [0]:
x_train = x_train/255
x_test = x_test/255

In [0]:
y_train = y_train.reshape([-1])
y_test = y_test.reshape([-1])

In [0]:
x = tf.placeholder(tf.float32, shape=[None,32,32,3])
y = tf.placeholder(tf.int64, shape=[None])

In [0]:
y_onehot = tf.one_hot(y,100)

In [0]:
keep_prob = tf.placeholder(tf.float32)

In [0]:
W_conv1 = tf.Variable(tf.truncated_normal(shape=[5,5,3,64], stddev = 5e-2))
b_conv1 = tf.Variable(tf.constant(0.1, shape = [64]))
h_conv1 = tf.nn.relu(tf.nn.conv2d(x,W_conv1,strides=[1,1,1,1],padding='SAME')+b_conv1)
h_pool1 = tf.nn.max_pool(h_conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [0]:
W_conv2 = tf.Variable(tf.truncated_normal(shape=[5,5,64,64], stddev= 5e-2))
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1,W_conv2,strides=[1,1,1,1],padding='SAME')+b_conv2)
h_pool2 = tf.nn.max_pool(h_conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')


In [0]:
W_fc1 = tf.Variable(tf.truncated_normal(shape=[8*8*64,384],stddev=5e-2))
b_fc1 = tf.Variable(tf.constant(0.1,shape=[384]))
h_pool2_flat = tf.reshape(h_pool2,[-1,8*8*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob = keep_prob)

In [0]:
W_fc2 = tf.Variable(tf.truncated_normal(shape=[384,100],stddev=5e-2))
b_fc2 = tf.Variable(tf.constant(0.1,shape=[100]))
logits = tf.matmul(h_fc1_drop,W_fc2)+b_fc2
y_pred = tf.nn.softmax(logits)

In [0]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_onehot,logits=logits))

In [0]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [0]:
correct_prediction = tf.equal(tf.argmax(y_pred,1),y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [55]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(10000):
    idx = np.random.randint(x_train.shape[0],size=128)
    batch = (x_train[idx],y_train[idx])
    if i% 100 == 0:
      train_accuracy = sess.run(accuracy,feed_dict={x:batch[0],y:batch[1],keep_prob:1.})
      loss_print = loss.eval(feed_dict={x:batch[0],y:batch[1],keep_prob:1.})
      print("step : %d, acc : %f, loss : %f" % (i,train_accuracy,loss_print))
    sess.run(train_step,feed_dict={x:batch[0],y:batch[1],keep_prob:0.5})  
  test_accuracy= accuracy.eval(feed_dict={x:x_test,y:y_test,keep_prob:1.})
  print("test acc: %f" % test_accuracy)

step : 0, acc : 0.023438, loss : 4.850376
step : 100, acc : 0.023438, loss : 4.489129
step : 200, acc : 0.046875, loss : 4.305403
step : 300, acc : 0.132812, loss : 4.034538
step : 400, acc : 0.085938, loss : 4.083412
step : 500, acc : 0.109375, loss : 3.943452
step : 600, acc : 0.250000, loss : 3.702430
step : 700, acc : 0.179688, loss : 3.634262
step : 800, acc : 0.140625, loss : 3.723573
step : 900, acc : 0.148438, loss : 3.648715
step : 1000, acc : 0.210938, loss : 3.457349
step : 1100, acc : 0.281250, loss : 3.380962
step : 1200, acc : 0.234375, loss : 3.317125
step : 1300, acc : 0.187500, loss : 3.435218
step : 1400, acc : 0.218750, loss : 3.435139
step : 1500, acc : 0.203125, loss : 3.392994
step : 1600, acc : 0.218750, loss : 3.406193
step : 1700, acc : 0.210938, loss : 3.175198
step : 1800, acc : 0.242188, loss : 3.177398
step : 1900, acc : 0.257812, loss : 3.093365
step : 2000, acc : 0.179688, loss : 3.240690
step : 2100, acc : 0.312500, loss : 3.105603
step : 2200, acc : 0.2